In [5]:
%pip install --upgrade webdriver-manager
#pip install lxml bs4 html5lib
#pip install --upgrade requests

Requirement already up-to-date: webdriver-manager in /opt/anaconda3/lib/python3.8/site-packages (3.5.2)
Note: you may need to restart the kernel to use updated packages.


In [1]:
#dependencies
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [2]:
#splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [3]:
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

In [4]:
longitude = driver.find_by_id('_x_id')
longitude.fill('-77.460317')

In [5]:
latitude = driver.find_by_id('_y_id')
latitude.fill('37.566379')

In [6]:
get_results = driver.find_by_id('proceed')
get_results.click()

In [7]:
html = driver.html
soup = BeautifulSoup(html, 'html.parser')
print(soup)

<html class="dj_webkit dj_chrome dj_contentbox js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths"><head><script type="text/javascript">
                    if (self === top) {
                        var frameBreak = document.getElementById("frameBreak");
                        frameBreak.parentNode.removeChild(frameBreak);
                    } else {
                        top.location = self.location;
                    }
                </script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=8" http-equiv="X-UA-Compatible"/><link h

In [9]:
results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
results
for result in results:
    result_text = result.text
    print(result_text)

Input:Longitude (X): -77.460317Latitude (Y): 37.566379Benchmark: Public_AR_Current (4)Vintage: Current_Current (4)Geographies: 2018 State Legislative Districts - Upper: STATE CODE: 51GEOID: 51010CENTLAT: +37.5400178AREAWATER: 14297325NAME: State Senate District 10CENTLON: -077.8342534AREALAND: 892249771States: STATE CODE: 51GEOID: 51CENTLAT: +37.5182631AREAWATER: 8528010384NAME: VirginiaCENTLON: -078.6759174AREALAND: 102258240496County Subdivisions: STATE CODE: 51COUSUB: 95291GEOID: 5176095291CENTLAT: +37.5294349COUNTY CODE: 760AREAWATER: 6849165NAME: Richmond cityCENTLON: -077.4755831AREALAND: 155205067Incorporated Places: STATE CODE: 51GEOID: 5167000CENTLAT: +37.5294349AREAWATER: 6849165PLACE: 67000NAME: Richmond cityPLACECC: C7CENTLON: -077.4755831AREALAND: 155205067PLACENS: 01789073Counties: STATE CODE: 51GEOID: 51760CENTLAT: +37.5294349COUNTY CODE: 760AREAWATER: 6849165NAME: Richmond cityCENTLON: -077.4755831AREALAND: 1552050672018 State Legislative Districts - Lower: STATE CODE: 

In [19]:
import re
result_digits = re.findall('\d+', result_text)
startswith = [i for i in result_digits if i.startswith('51')]
fips_elements =[element for element in startswith if len(element)>=15]
print(result_digits)
print(startswith)
print(fips_elements)

['77', '460317', '37', '566379', '4', '4', '2018', '51', '51010', '37', '5400178', '14297325', '10', '077', '8342534', '892249771', '51', '51', '37', '5182631', '8528010384', '078', '6759174', '102258240496', '51', '95291', '5176095291', '37', '5294349', '760', '6849165', '077', '4755831', '155205067', '51', '5167000', '37', '5294349', '6849165', '67000', '7', '077', '4755831', '155205067', '01789073', '51', '51760', '37', '5294349', '760', '6849165', '077', '4755831', '1552050672018', '51', '51071', '37', '5598292', '539858', '71', '077', '4329844', '42130930116', '51', '5104', '36', '9624589', '296493194', '4', '077', '1571007', '94332173982020', '51', '517600402022018', '37', '5669913', '760', '0', '2018', '077', '4562910', '040202', '157365', '2018', '51', '51760040202', '37', '5669600', '760', '0', '402', '02', '077', '4639641', '040202', '3556654']
['51', '51010', '51', '51', '5182631', '51', '5176095291', '51', '5167000', '51', '51760', '51', '51071', '51', '5104', '51', '517600

In [333]:
fips_check = pd.read_csv("data/state_data/geo/geo_va.csv")
fips_check.head()
fips_check.columns

Index(['Unnamed: 0', 'index', 'LoanNumber', 'DateApproved', 'SBAOfficeCode',
       'ProcessingMethod', 'BorrowerName', 'BorrowerAddress', 'BorrowerCity',
       'BorrowerState', 'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'Originat

In [338]:
columns = ['Unnamed: 0', 'index', 'LoanNumber','full_add', 'Lat', 'Long',]
fips_check = fips_check[columns]
fips_check_2 =fips_check.iloc[22311:]
fips_check_2

,Unnamed: 0,index,LoanNumber,full_add,Lat,Long
22311,22311,918429,2151538602,"5300 Airport Square Ln, Sandston, VA",37.519462,-77.340384
22312,22312,918430,3783977209,"589 WILLOW ROSE DR, MC GAHEYSVILLE, VA",38.374799,-78.723153
22313,22313,918431,2911888410,"1501 Avon Street Extended 1501 Avon Street Extended, Charlottesville, VA",38.014393,-78.484909
22314,22314,918432,8941057007,"9544 Burke Rd, BURKE, VA",38.795417,-77.271548
22315,22315,918433,7060798400,"220 Imboden Dr, Winchester, VA",39.194545,-78.136088
...,...,...,...,...,...,...
23263,23263,919381,9512327001,"6000 FORT AVE, LYNCHBURG, VA",37.371059,-79.194838
23264,23264,919382,9677417002,"3337 Emmaus Rd, ROCKINGHAM, VA",38.406665,-78.913133
23265,23265,919383,9715167306,"8503 BUCYRUS CT, MANASSAS, VA",38.761194,-77.454184
23266,23266,919384,9767568404,"554 E Mercury Blvd, Hampton, VA",37.022287,-76.319882


In [339]:
import re
lat = fips_check_2["Lat"].astype(float).astype(str).to_list()
long = fips_check_2["Long"].astype(float).astype(str).to_list()

In [340]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('51')]
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


0,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
1,38.374799100000004,-78.72315309999999 is ['511650119001009']
2,38.0143935,-78.4849087 is ['510030113031000']
3,38.7954166,-77.2715478 is ['510594309021000']
4,39.1945447,-78.136088 is ['510690511022010']
5,38.383656200000004,-78.9743824 is ['511650115004010']
6,37.303200200000006,-77.29495990000001 is ['516708203002034']
7,39.1945447,-78.136088 is ['510690511022010']
8,36.7731429,-76.580919 is ['518000754024000']
9,38.808446999999994,-77.219898 is ['510594304003023']
10,37.131703200000004,-81.5272725 is ['511850203012031']
11,36.8493191,-76.290651 is ['517100049003053']
12,38.9658393,-77.3729967 is ['510594808022012']
13,38.8091547,-78.5919215 is ['511710406001018']
14,36.9489207,-79.9520146 is ['510670207011024']
15,38.886349700000004,-77.0952071 is ['510131019001001']
16,37.118118599999995,-76.484524 is ['517000321302005']
17,38.897941100000004,-78.49157679999999 is ['511710405012002']
18,36.774483000000004,-

152,37.3336346,-80.75132109999998 is ['510719303006064']
153,37.9895066,-77.75998249999999 is ['511099501011035']
154,39.0570377,-77.4990321 is ['511076110121015']
155,37.5371976,-77.51879620000001 is ['517600704001020']
156,37.41442,-76.5296074 is ['510731002033017']
157,38.8899252,-77.4566622 is ['510594901042036']
158,38.0687065,-78.8889714 is ['518200031001065']
159,38.9222519,-77.2335635 is ['510594802052010']
160,37.6521153,-77.5831983 is ['510872001503007']
161,38.9576221,-77.3333619 is ['510594819001002']
162,39.161277,-78.16657049999999 is ['518400003024008']
163,37.5517213,-77.4568734 is ['517600404002001']
164,39.0451141,-77.52291020000001 is ['511076110041108']
165,37.3597939,-77.6442608 is ['510411010141008']
166,38.9469257,-77.3441753 is ['510594823013000']
167,39.117664700000006,-77.54702950000001 is ['511076105044006']
168,38.036549,-78.4832305 is ['515400010002014']
169,38.7653322,-77.5217679 is ['511539014091005']
170,36.778079299999995,-76.1916188 is ['51810046219100

303,37.2653218,-79.976838 is ['517700018004004']
304,38.871838000000004,-77.369292 is ['510594826031012']
305,37.4411976,-79.11584659999998 is ['510090105042004']
306,38.760861,-77.454151 is ['516839103011000']
307,38.8136855,-77.04338680000001 is ['515102018051000']
308,37.639866,-77.488167 is ['510872008011012']
309,38.7843881,-77.55440420000001 is ['511539014095001']
310,37.4411976,-79.11584659999998 is ['510090105042004']
311,38.037113399999996,-78.48811409999999 is ['515400002012022']
312,37.608795,-77.513762 is ['510872004041009']
313,37.1125071,-76.46222829999999 is ['511990503051018']
314,38.9083561,-77.53663890000001 is ['511076118081007']
315,38.8958461,-77.0689353 is ['510131016033001']
316,38.8958461,-77.0689353 is ['510131016033001']
317,38.8708635,-77.3669036 is ['510594826031011']
318,38.430739700000004,-78.91348620000001 is ['511650114005003']
319,37.3519494,-77.39977090000001 is ['510411005092003']
320,38.8795276,-77.237372 is ['510594616053001']
321,36.6762849,-82.385

453,36.8370864,-76.1245887 is ['518100456051003']
454,38.4303484,-78.8483061 is ['516600002053005']
455,38.8898153,-77.45673000000001 is ['510594901042036']
456,38.840558,-77.4407302 is ['510594912021004']
457,37.5748184,-77.4750819 is ['517600501001001']
458,37.5748184,-77.4750819 is ['517600501001001']
459,36.82056179999999,-76.1181645 is ['518100458071000']
460,37.5481575,-77.4618379 is ['517600411002000']
461,38.8718175,-77.2074838 is ['510594501002000']
462,38.8827602,-77.1714532 is ['516105001001030']
463,38.769665,-77.4867359 is ['516839102021016']
464,36.846045000000004,-76.2881059 is ['517100049003069']
465,36.846045000000004,-76.2881059 is ['517100049003069']
466,38.867098799999994,-77.2386742 is ['510594402013004']
467,38.0156068,-78.4703547 is ['515400004023013']
468,37.314532299999996,-79.18582430000001 is ['510310202001036']
469,37.0793612,-77.99498709999999 is ['511350001023041']
470,38.9690278,-77.48839559999999 is ['511076119013039']
471,38.883815000000006,-77.43891190

605,38.809262,-77.2575426 is ['510594318021000']
606,38.4669206,-77.40434470000001 is ['511790102164001']
607,37.28396420000001,-80.07571159999999 is ['517750105022007']
608,37.5519235,-77.478062 is ['517600409002012']
609,37.536446600000005,-77.43357359999999 is ['517600305021076']
610,37.6647684,-77.61271740000001 is ['510872001491002']
611,38.022037100000006,-78.46729090000001 is ['515400004023000']
612,38.0630791,-78.4897788 is ['515400008004003']
613,37.2623424,-79.9379588 is ['517700027002019']
614,38.7708425,-77.4433497 is ['516859202011004']
615,38.8936052,-77.0725453 is ['510131017053003']
616,38.7425231,-77.0567387 is ['510594155001019']
617,38.7425231,-77.0567387 is ['510594155001019']
618,36.843121999999994,-76.02320329999999 is ['518100448063002']
619,37.1496896,-82.4645114 is ['510510403003010']
620,37.3570383,-77.9594541 is ['510079302002011']
621,38.898746100000004,-77.33482049999999 is ['510594816002003']
622,37.03871229999999,-76.3879767 is ['516500105023012']
623,36.

757,38.042440500000005,-78.51334920000001 is ['515400007004001']
758,37.9052802,-78.8477974 is ['511259502011049']
759,38.545426500000005,-78.92882420000001 is ['511650111002004']
760,38.078719899999996,-78.47814220000001 is ['510030107021000']
761,37.223258,-79.985763 is ['511610308032031']
762,37.687847999999995,-77.594916 is ['510872001482000']
763,37.3389975,-75.921974 is ['511319302004096']
764,38.772013799999996,-77.43931529999999 is ['516859202011001']
765,38.9255456,-77.2731783 is ['510594603002000']
766,37.4065109,-77.66494499999999 is ['510411010073006']
767,37.945938500000004,-76.39568390000001 is ['511330202001053']
768,38.804835499999996,-77.0469214 is ['515102019001015']
769,38.716148,-77.19644699999999 is ['510594220002018']
770,37.1314171,-81.4963704 is ['511850211011004']
771,38.966806,-77.73912790000001 is ['511076109001042']
772,37.945938500000004,-76.39568390000001 is ['511330202001053']
773,38.8654218,-77.281685 is ['516003002001004']
774,38.7283243,-77.5722854 is 

907,38.8435875,-77.311802 is ['516003004001012']
908,36.870117799999996,-76.2772828 is ['517100029001026']
909,38.8900404,-77.0866741 is ['510131018051008']
910,38.892695200000006,-77.0812066 is ['510131017012000']
911,37.4988456,-77.5324734 is ['510411002101011']
912,37.607454600000004,-78.7004736 is ['510299302022000']
913,38.713449,-77.794201 is ['510619304051027']
914,37.2826802,-79.99068879999999 is ['517700009002000']
915,37.5447907,-77.44122709999999 is ['517600305011002']
916,38.742217100000005,-77.4991072 is ['516839104024006']
917,37.362514000000004,-79.1809093 is ['516800016011000']
918,37.3879005,-76.75365390000002 is ['510950804042011']
919,37.502187400000004,-77.60657420000001 is ['510411009222012']
920,36.9878113,-82.6358683 is ['511959310001027']
921,38.928743299999994,-77.245275 is ['510594604003002']
922,36.7780359,-76.232477 is ['515500208053019']
923,38.789604700000005,-77.54563409999999 is ['511539014182004']
924,39.1562614,-78.1569824 is ['510690509001020']
925,38

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [34]:
#fips_fill = fips_check.iloc[:19651]
#fips_fill

,Unnamed: 0,index,LoanNumber,full_add,Lat,Long
0,0,896118,2392497107,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486
1,1,896119,2649457102,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746
2,2,896120,2952437110,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711
3,3,896121,3379937304,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008
4,4,896122,3773127208,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420
...,...,...,...,...,...,...
19646,19646,915764,6486717710,"2200 OPITZ BLVD STE 200, WOODBRIDGE, VA",38.633430,-77.279425
19647,19647,915765,6123547305,"1775 Tysons Boulevard, 5th Floor, Tysons, VA",38.853983,-77.051499
19648,19648,915766,2480967106,"3404 Westminster Ave, NORFOLK, VA",36.843798,-76.244896
19649,19649,915767,2782827301,"60 ROCK POINTE LN, WARRENTON, VA",38.723874,-77.791469


In [53]:
fips_fill_2 = fips_check_2
fips_fill_2

,Unnamed: 0,index,LoanNumber,full_add,Lat,Long
19651,19651,915769,5692578000,"7204 Harrison Lane, Alexandria, VA",38.761272,-77.095430
19652,19652,915770,6809717106,"2800 S SHIRLINGTON RD, ARLINGTON, VA",38.842551,-77.084576
19653,19653,915771,8219548403,"3404 Westminster Ave, Norfolk, VA",36.843798,-76.244896
19654,19654,915772,8295697201,"100 Globe RD, Aylett, VA",37.793870,-77.183643
19655,19655,915773,9070597406,"14225 Penrose Pl 100B, Chantilly, VA",38.893904,-77.444724
...,...,...,...,...,...,...
23263,23263,919381,9512327001,"6000 FORT AVE, LYNCHBURG, VA",37.371059,-79.194838
23264,23264,919382,9677417002,"3337 Emmaus Rd, ROCKINGHAM, VA",38.406665,-78.913133
23265,23265,919383,9715167306,"8503 BUCYRUS CT, MANASSAS, VA",38.761194,-77.454184
23266,23266,919384,9767568404,"554 E Mercury Blvd, Hampton, VA",37.022287,-76.319882


In [341]:
geo_id # test example from fips_check.iloc[22311:]

[['519462299999994', '510872014061012'],
 ['511650119001009'],
 ['510030113031000'],
 ['510594309021000'],
 ['510690511022010'],
 ['511650115004010'],
 ['516708203002034'],
 ['510690511022010'],
 ['518000754024000'],
 ['510594304003023'],
 ['511850203012031'],
 ['517100049003053'],
 ['510594808022012'],
 ['511710406001018'],
 ['510670207011024'],
 ['510131019001001'],
 ['517000321302005'],
 ['511710405012002'],
 ['515500214032006'],
 ['511850211022002'],
 ['510411009222014'],
 ['511076118134000'],
 ['511076115013022'],
 ['510594220002002'],
 ['511539002021008'],
 ['510030108021004'],
 ['511430106002037'],
 ['516839101003012'],
 ['517000321283051'],
 ['516839101003012'],
 ['511076107012014'],
 ['510131014063005'],
 ['518100454072022'],
 ['510731004001032'],
 ['510690506003003'],
 ['511770201082008'],
 ['510310201011025'],
 ['517700009001003'],
 ['516105002001001'],
 ['510594707001005'],
 ['510594825051006'],
 ['510690504021067'],
 ['510030107023006'],
 ['510594525011000'],
 ['5105945250

In [359]:
#geo_id2 = [i[0] for i in geo_id]

In [297]:
#fips_fill['FIPS_z'] = geo_id
fips_loc = fips_fill
#coordinates with more than one return:
test = fips_loc[fips_loc['FIPS_z'].str.len() > 1]
test[['FIPS_x', 'FIPS_z']]  = test['FIPS_z'].apply(lambda x: pd.Series(str(x).split(",")))
test = test.drop('FIPS_x', axis=1)
test['FIPS_z']= test['FIPS_z'].replace(']', '', regex=True)
test['FIPS_z']= test['FIPS_z'].replace(' ', '', regex=True)
test['FIPS_z']= test['FIPS_z'].replace("'", '', regex=True)
cols = ['LoanNumber', 'FIPS_z']
test = test[cols]
fill_merge = fips_fill.merge(test, how="left", on="LoanNumber")
fill_merge


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Unnamed: 0,index,LoanNumber,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,896118,2392497107,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,[515500203001000],NaN
1,1,896119,2649457102,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,[510872005033029],NaN
2,2,896120,2952437110,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,[510411001071014],NaN
3,3,896121,3379937304,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,[511076104001030],NaN
4,4,896122,3773127208,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,[518100430061015],NaN
...,...,...,...,...,...,...,...,...
19646,19646,915764,6486717710,"2200 OPITZ BLVD STE 200, WOODBRIDGE, VA",38.633430,-77.279425,[511539005032000],NaN
19647,19647,915765,6123547305,"1775 Tysons Boulevard, 5th Floor, Tysons, VA",38.853983,-77.051499,[510131034052000],NaN
19648,19648,915766,2480967106,"3404 Westminster Ave, NORFOLK, VA",36.843798,-76.244896,[517100046001005],NaN
19649,19649,915767,2782827301,"60 ROCK POINTE LN, WARRENTON, VA",38.723874,-77.791469,[510619303052016],NaN


In [304]:
fill_merge['FIPS_z_x'] = fill_merge['FIPS_z_x'].str[0]
fill_merge['FIPS_z'] = fill_merge['FIPS_z_y'].fillna(fill_merge['FIPS_z_x'])
fill_merge = fill_merge.drop(['FIPS_z_x'], axis=1)
fill_merge = fill_merge.drop(['FIPS_z_y'], axis=1)
fill_merge

,Unnamed: 0,index,LoanNumber,full_add,Lat,Long,FIPS_z
0,0,896118,2392497107,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,515500203001000
1,1,896119,2649457102,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,510872005033029
2,2,896120,2952437110,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,510411001071014
3,3,896121,3379937304,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,511076104001030
4,4,896122,3773127208,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,518100430061015
...,...,...,...,...,...,...,...
19646,19646,915764,6486717710,"2200 OPITZ BLVD STE 200, WOODBRIDGE, VA",38.633430,-77.279425,511539005032000
19647,19647,915765,6123547305,"1775 Tysons Boulevard, 5th Floor, Tysons, VA",38.853983,-77.051499,510131034052000
19648,19648,915766,2480967106,"3404 Westminster Ave, NORFOLK, VA",36.843798,-76.244896,517100046001005
19649,19649,915767,2782827301,"60 ROCK POINTE LN, WARRENTON, VA",38.723874,-77.791469,510619303052016


In [357]:
fips_fill_2['FIPS_z'] = geo_id
fips_fill_2['FIPS_z'] = fips_fill_2['FIPS_z'].str[0]
fips_fill_2

,Unnamed: 0,index,LoanNumber,full_add,Lat,Long,FIPS_z
19651,19651,915769,5692578000,"7204 Harrison Lane, Alexandria, VA",38.761272,-77.095430,510594213002002
19652,19652,915770,6809717106,"2800 S SHIRLINGTON RD, ARLINGTON, VA",38.842551,-77.084576,515102010003008
19653,19653,915771,8219548403,"3404 Westminster Ave, Norfolk, VA",36.843798,-76.244896,517100046001005
19654,19654,915772,8295697201,"100 Globe RD, Aylett, VA",37.793870,-77.183643,511019501022040
19655,19655,915773,9070597406,"14225 Penrose Pl 100B, Chantilly, VA",38.893904,-77.444724,510594901042034
...,...,...,...,...,...,...,...
23263,23263,919381,9512327001,"6000 FORT AVE, LYNCHBURG, VA",37.371059,-79.194838,516800009013012
23264,23264,919382,9677417002,"3337 Emmaus Rd, ROCKINGHAM, VA",38.406665,-78.913133,511650114005035
23265,23265,919383,9715167306,"8503 BUCYRUS CT, MANASSAS, VA",38.761194,-77.454184,516839103011000
23266,23266,919384,9767568404,"554 E Mercury Blvd, Hampton, VA",37.022287,-76.319882,516500109001011


In [351]:
frames = [fill_merge, fips_fill_2]
result = pd.concat(frames)

In [354]:
result

,Unnamed: 0,index,LoanNumber,full_add,Lat,Long,FIPS_z
0,0,896118,2392497107,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,515500203001000
1,1,896119,2649457102,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,510872005033029
2,2,896120,2952437110,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,510411001071014
3,3,896121,3379937304,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,511076104001030
4,4,896122,3773127208,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,518100430061015
...,...,...,...,...,...,...,...
23263,23263,919381,9512327001,"6000 FORT AVE, LYNCHBURG, VA",37.371059,-79.194838,516800009013012
23264,23264,919382,9677417002,"3337 Emmaus Rd, ROCKINGHAM, VA",38.406665,-78.913133,511650114005035
23265,23265,919383,9715167306,"8503 BUCYRUS CT, MANASSAS, VA",38.761194,-77.454184,516839103011000
23266,23266,919384,9767568404,"554 E Mercury Blvd, Hampton, VA",37.022287,-76.319882,516500109001011


In [307]:
result_cols = ["LoanNumber", "FIPS_z"]
result_parsed = result[result_cols]
result_parsed

,LoanNumber,FIPS_z
0,2392497107,515500203001000
1,2649457102,510872005033029
2,2952437110,510411001071014
3,3379937304,511076104001030
4,3773127208,518100430061015
...,...,...
23263,9512327001,516800009013012
23264,9677417002,511650114005035
23265,9715167306,516839103011000
23266,9767568404,516500109001011


In [308]:
result_parsed.iloc[33:]

,LoanNumber,FIPS_z
33,6471457701,511790102011041
34,1124547108,510594823013000
35,9515788600,511076112081014
36,7444697005,518100456032018
37,5663597708,515102019002001
...,...,...
23263,9512327001,516800009013012
23264,9677417002,511650114005035
23265,9715167306,516839103011000
23266,9767568404,516500109001011


In [24]:
#merge fips results with main dataframe
geo_va = pd.read_csv("data/state_data/geo/geo_va.csv")
geo_va

,Unnamed: 0,index,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Estimated_LenderPercent,Estimated_LenderProfit,DateApproved_ts,ForgivenessDate_ts,Industry,SBAOfficeLabel,Loan_Range,full_add,Lat,Long
0,0,896118,2392497107,4/10/20,304,PPP,TECNICO CORPORATION,831 INDUSTRIAL AVE,CHESAPEAKE,VA,23324-2614,10/19/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,225134,Truist Bank d/b/a Branch Banking & Trust Co,214 N Tryon St,CHARLOTTE,NC,28202-1078,U,Y,Y,Existing or more than 2 years old,CHESAPEAKE,CHESAPEAKE CITY,VA,23324-2614,VA-03,500.0,336611.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,225134,Truist Bank d/b/a Branch Banking & Trust Co,CHARLOTTE,NC,Unanswered,Unanswered,NaN,10141944.44,9/20/21,33.0,1%,100000.0,1586476800,1632096000,Manufacturing,RICHMOND DISTRICT OFFICE,e $5-10 million,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486
1,1,896119,2649457102,4/11/20,304,PPP,"FIRE & LIFE SAFETY AMERICA, INC.",8827 STAPLES MILL RD,HENRICO,VA,23228-2014,10/21/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,60009,First Commonwealth Bank,600 Philadelphia St,INDIANA,PA,15701-3904,U,N,N,Existing or more than 2 years old,HENRICO,HENRICO,VA,23228-2014,VA-07,500.0,922160.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,60009,First Commonwealth Bank,INDIANA,PA,Unanswered,Unanswered,NaN,10143835.62,9/28/21,92.0,1%,100000.0,1586563200,1632787200,Public Administration,RICHMOND DISTRICT OFFICE,e $5-10 million,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746
2,2,896120,2952437110,4/11/20,304,PPP,"TIMMONS GROUP, INC.",1001 Boulders Parkway Ste 300,RICHMOND,VA,23225-5505,8/5/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,123548,Towne Bank,5716 High St West,PORTSMOUTH,VA,23703-4502,U,Y,Y,Existing or more than 2 years old,RICHMOND,RICHMOND CITY,VA,23225-5505,VA-07,500.0,541330.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,123548,Towne Bank,PORTSMOUTH,VA,Unanswered,Unanswered,NaN,10124722.22,7/14/21,54.0,1%,100000.0,1586563200,1626220800,"Professional, Scientific, and Technical Services",RICHMOND DISTRICT OFFICE,e $5-10 million,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711
3,3,896121,3379937304,4/29/20,353,PPP,LOUDOUN MEDICAL GROUP PC,224 D Cornwall Street Suite 403,LEESBURG,VA,20176,10/7/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,44449,"PNC Bank, National Association",222 Delaware Ave,WILMINGTON,DE,19801-1621,U,N,N,Unanswered,LEESBURG,LOUDOUN,VA,20176-1200,VA-10,500.0,621112.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,44449,"PNC Bank, National Association",WILMINGTON,DE,Unanswered,Unanswered,NaN,10137777.78,9/10/21,62.0,1%,100000.0,1588118400,1631232000,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,e $5-10 million,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008
4,4,896122,3773127208,4/27/20,304,PPP,BURGERBUSTERS INC,2242 W GREAT NECK RD,VIRGINIA BEACH,VA,23451,7/21/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,44449,"PNC Bank, National Association",222 Delaware Ave,WILMINGTON,DE,19801-1621,U,N,N,Existing or more than 2 years old,V

In [ ]:
geo_va_fips = geo_va.merge(result_parsed, how="left", on="LoanNumber")

In [20]:
#checking columns with multiple FIPS, for 2nds batch 19651:
#replace FIPS
geo_va_fips.loc[geo_va_fips['FIPS_z']=='514070000000004', 'FIPS_z']= '510872015041003'
geo_va_fips.loc[geo_va_fips['FIPS_z']=='516454700000004', 'FIPS_z']= '511455001022014'
geo_va_fips.loc[geo_va_fips['FIPS_z']=='519462299999994', 'FIPS_z']= '510872014061012'
geo_va_fips.head()

,Unnamed: 0,index,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Estimated_LenderPercent,Estimated_LenderProfit,DateApproved_ts,ForgivenessDate_ts,Industry,SBAOfficeLabel,Loan_Range,full_add,Lat,Long,FIPS_z
0,0,896118,2392497107,4/10/20,304,PPP,TECNICO CORPORATION,831 INDUSTRIAL AVE,CHESAPEAKE,VA,23324-2614,10/19/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,225134,Truist Bank d/b/a Branch Banking & Trust Co,214 N Tryon St,CHARLOTTE,NC,28202-1078,U,Y,Y,Existing or more than 2 years old,CHESAPEAKE,CHESAPEAKE CITY,VA,23324-2614,VA-03,500.0,336611.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,225134,Truist Bank d/b/a Branch Banking & Trust Co,CHARLOTTE,NC,Unanswered,Unanswered,NaN,10141944.44,9/20/21,33.0,1%,100000.0,1586476800,1632096000,Manufacturing,RICHMOND DISTRICT OFFICE,e $5-10 million,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.8243,-76.2725,5.155e+14
1,1,896119,2649457102,4/11/20,304,PPP,"FIRE & LIFE SAFETY AMERICA, INC.",8827 STAPLES MILL RD,HENRICO,VA,23228-2014,10/21/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,60009,First Commonwealth Bank,600 Philadelphia St,INDIANA,PA,15701-3904,U,N,N,Existing or more than 2 years old,HENRICO,HENRICO,VA,23228-2014,VA-07,500.0,922160.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,60009,First Commonwealth Bank,INDIANA,PA,Unanswered,Unanswered,NaN,10143835.62,9/28/21,92.0,1%,100000.0,1586563200,1632787200,Public Administration,RICHMOND DISTRICT OFFICE,e $5-10 million,"8827 STAPLES MILL RD, HENRICO, VA",37.6339,-77.5077,5.10872e+14
2,2,896120,2952437110,4/11/20,304,PPP,"TIMMONS GROUP, INC.",1001 Boulders Parkway Ste 300,RICHMOND,VA,23225-5505,8/5/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,123548,Towne Bank,5716 High St West,PORTSMOUTH,VA,23703-4502,U,Y,Y,Existing or more than 2 years old,RICHMOND,RICHMOND CITY,VA,23225-5505,VA-07,500.0,541330.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,123548,Towne Bank,PORTSMOUTH,VA,Unanswered,Unanswered,NaN,10124722.22,7/14/21,54.0,1%,100000.0,1586563200,1626220800,"Professional, Scientific, and Technical Services",RICHMOND DISTRICT OFFICE,e $5-10 million,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.5111,-77.5287,5.10411e+14
3,3,896121,3379937304,4/29/20,353,PPP,LOUDOUN MEDICAL GROUP PC,224 D Cornwall Street Suite 403,LEESBURG,VA,20176,10/7/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,44449,"PNC Bank, National Association",222 Delaware Ave,WILMINGTON,DE,19801-1621,U,N,N,Unanswered,LEESBURG,LOUDOUN,VA,20176-1200,VA-10,500.0,621112.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,44449,"PNC Bank, National Association",WILMINGTON,DE,Unanswered,Unanswered,NaN,10137777.78,9/10/21,62.0,1%,100000.0,1588118400,1631232000,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,e $5-10 million,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.1194,-77.569,5.11076e+14
4,4,896122,3773127208,4/27/20,304,PPP,BURGERBUSTERS INC,2242 W GREAT NECK RD,VIRGINIA BEACH,VA,23451,7/21/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,44449,"PNC Bank, National Association",222 Delaware Ave,WILMINGTON,DE,19801-1621,U,N,N

In [73]:
null_check = geo_va_fips.loc[geo_va_fips['FIPS_z'].isnull()] #FIPS null check, 33, review addresses 
null_check.head()
# ("data/state_data/geo/geo_fips/va_fips_scraped_null.csv") -export to census batch address
# ("data/state_data/geo/geo_fips/va_fips_scraped_null_results.csv") 

null_read = pd.read_csv("data/state_data/geo/geo_fips/va_fips_scraped_null_results.csv") 
cols = ["full_add", "new_add", "Long", "Lat"]
null_read = null_read[cols]
null_read = null_read[null_read["Long"].notna()]
null_read

,full_add,new_add,Long,Lat
0,"7918 Jones Branch Drive, Suite 300, McLean, VA","7918 JONES BRANCH DR, MCLEAN, VA, 22102",-77.217060,38.927105
2,"7918 Jones Branch Dr Ste 300, McLean, VA","7918 JONES BRANCH DR, MCLEAN, VA, 22102",-77.217060,38.927105
3,"5534 North MAIN ST, MOUNT JACKSON, VA","5534 S MAIN ST, MOUNT JACKSON, VA, 22842",-78.633680,38.755116
5,"8200 GREENSBORO DR STE 900, MC LEAN, VA","8200 GREENSBORO DR, MC LEAN, VA, 22102",-77.228676,38.922337
6,"8200 Greensboro Dr, Suite 900, MC LEAN, VA","8200 GREENSBORO DR, MC LEAN, VA, 22102",-77.228676,38.922337
7,"5534 N MAIN ST, MOUNT JACKSON, VA","5534 S MAIN ST, MOUNT JACKSON, VA, 22842",-78.633690,38.755110
8,"5534 North MAIN ST, MOUNT JACKSON, VA","5534 S MAIN ST, MOUNT JACKSON, VA, 22842",-78.633680,38.755116
9,"8200 Greensboro Drive, Ste 900, McLean, VA","8200 GREENSBORO DR, MCLEAN, VA, 22102",-77.228676,38.922337
10,"8200 Greensboro Dr Ste 900, McLean, VA","8200 GREENSBORO DR, MCLEAN, VA, 22102",-77.228676,38.922337
13,"2300 9th Street PH3, ARLINGTON, VA","2300 9TH ST S, ARLINGTON, VA, 22204",-77.082560,38.864834


In [74]:
geo_va_fips = pd.read_csv("data/state_data/geo/geo_fips/va_fips_scraped.csv") 
geo_va_fips = geo_va_fips.merge(null_read.drop_duplicates(subset=['full_add']), on="full_add", how="left")
geo_va_fips['Lat'] = geo_va_fips['Lat_y'].fillna(geo_va_fips['Lat_x'])
geo_va_fips['Long'] = geo_va_fips['Long_y'].fillna(geo_va_fips['Long_x'])
geo_va_fips = geo_va_fips.drop(['Lat_y', 'Lat_x'], axis=1)
geo_va_fips = geo_va_fips.drop(['Long_y', 'Long_x'], axis=1)
geo_va_fips['full_add'] = geo_va_fips['new_add'].fillna(geo_va_fips['full_add'])
geo_va_fips = geo_va_fips.drop(['new_add'], axis=1)
geo_va_fips #get FIPS_z for NaN

,Unnamed: 0,Unnamed: 0.1,index,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Estimated_LenderPercent,Estimated_LenderProfit,DateApproved_ts,ForgivenessDate_ts,Industry,SBAOfficeLabel,Loan_Range,full_add,FIPS_z,Lat,Long
0,0,0,896118,2392497107,4/10/20,304,PPP,TECNICO CORPORATION,831 INDUSTRIAL AVE,CHESAPEAKE,VA,23324-2614,10/19/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,225134,Truist Bank d/b/a Branch Banking & Trust Co,214 N Tryon St,CHARLOTTE,NC,28202-1078,U,Y,Y,Existing or more than 2 years old,CHESAPEAKE,CHESAPEAKE CITY,VA,23324-2614,VA-03,500.0,336611.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,225134,Truist Bank d/b/a Branch Banking & Trust Co,CHARLOTTE,NC,Unanswered,Unanswered,NaN,10141944.44,9/20/21,33.0,1%,100000.0,1586476800,1632096000,Manufacturing,RICHMOND DISTRICT OFFICE,e $5-10 million,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",5.155002e+14,36.824257,-76.272486
1,1,1,896119,2649457102,4/11/20,304,PPP,"FIRE & LIFE SAFETY AMERICA, INC.",8827 STAPLES MILL RD,HENRICO,VA,23228-2014,10/21/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,60009,First Commonwealth Bank,600 Philadelphia St,INDIANA,PA,15701-3904,U,N,N,Existing or more than 2 years old,HENRICO,HENRICO,VA,23228-2014,VA-07,500.0,922160.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,60009,First Commonwealth Bank,INDIANA,PA,Unanswered,Unanswered,NaN,10143835.62,9/28/21,92.0,1%,100000.0,1586563200,1632787200,Public Administration,RICHMOND DISTRICT OFFICE,e $5-10 million,"8827 STAPLES MILL RD, HENRICO, VA",5.108720e+14,37.633918,-77.507746
2,2,2,896120,2952437110,4/11/20,304,PPP,"TIMMONS GROUP, INC.",1001 Boulders Parkway Ste 300,RICHMOND,VA,23225-5505,8/5/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,123548,Towne Bank,5716 High St West,PORTSMOUTH,VA,23703-4502,U,Y,Y,Existing or more than 2 years old,RICHMOND,RICHMOND CITY,VA,23225-5505,VA-07,500.0,541330.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,123548,Towne Bank,PORTSMOUTH,VA,Unanswered,Unanswered,NaN,10124722.22,7/14/21,54.0,1%,100000.0,1586563200,1626220800,"Professional, Scientific, and Technical Services",RICHMOND DISTRICT OFFICE,e $5-10 million,"1001 Boulders Parkway Ste 300, RICHMOND, VA",5.104110e+14,37.511077,-77.528711
3,3,3,896121,3379937304,4/29/20,353,PPP,LOUDOUN MEDICAL GROUP PC,224 D Cornwall Street Suite 403,LEESBURG,VA,20176,10/7/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,44449,"PNC Bank, National Association",222 Delaware Ave,WILMINGTON,DE,19801-1621,U,N,N,Unanswered,LEESBURG,LOUDOUN,VA,20176-1200,VA-10,500.0,621112.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,44449,"PNC Bank, National Association",WILMINGTON,DE,Unanswered,Unanswered,NaN,10137777.78,9/10/21,62.0,1%,100000.0,1588118400,1631232000,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,e $5-10 million,"224 D Cornwall Street Suite 403, LEESBURG, VA",5.110761e+14,39.119430,-77.569008
4,4,4,896122,3773127208,4/27/20,304,PPP,BURGERBUSTERS INC,2242 W GREAT NECK RD,VIRGINIA BEACH,VA,23451,7/21/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,44449,"PNC Bank, National Association",2

In [79]:
import re
lat = null_check["Lat"].astype(float).astype(str).to_list()
long = null_check["Long"].astype(float).astype(str).to_list()
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('51')]
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


0,38.927105,-77.21706 is ['510594802043017']
1,35.8824695,-78.8466994 is []
2,38.927105,-77.21706 is ['510594802043017']
3,38.755115999999994,-78.63368 is ['511710407001010']
4,38.89937000000001,-77.01959000000002 is []
5,38.922337,-77.22867600000001 is ['510594802052005']
6,38.922337,-77.22867600000001 is ['510594802052005']
7,38.755109999999995,-78.63369 is ['511710407001010']
8,38.755115999999994,-78.63368 is ['511710407001010']
9,38.922337,-77.22867600000001 is ['510594802052005']
10,38.922337,-77.22867600000001 is ['510594802052005']
11,39.1817782,-76.61551949999998 is []
12,44.4512216,-73.19911429999998 is []
13,38.864834,-77.08256 is ['510131025002010']
14,34.5060509,-93.0198245 is []
15,38.922337,-77.22867600000001 is ['510594802052005']
16,38.96373970000001,-76.5491392 is []
17,30.4791203,-97.8029323 is []
18,30.4791203,-97.8029323 is []
19,38.755115999999994,-78.63368 is ['511710407001010']
20,38.755115999999994,-78.63368 is ['511710407001010']
21,28.0394401,-82.5369452 is []

In [91]:
null_check["FIPS_z"]= geo_id
null_check["FIPS_z"]= null_check["FIPS_z"].str[0]
cols = ["LoanNumber", "FIPS_z"]
null_check = null_check[cols]
geo_va_fips = geo_va_fips.merge(null_check, on="LoanNumber", how="left")
geo_va_fips["FIPS_z"] = geo_va_fips["FIPS_z_y"].fillna(geo_va_fips["FIPS_z_x"])
geo_va_fips = geo_va_fips.drop("FIPS_z_x", axis=1)
geo_va_fips = geo_va_fips.drop("FIPS_z_y", axis=1)
geo_va_fips

,Unnamed: 0,Unnamed: 0.1,index,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Estimated_LenderPercent,Estimated_LenderProfit,DateApproved_ts,ForgivenessDate_ts,Industry,SBAOfficeLabel,Loan_Range,full_add,Lat,Long,FIPS_z
0,0,0,896118,2392497107,4/10/20,304,PPP,TECNICO CORPORATION,831 INDUSTRIAL AVE,CHESAPEAKE,VA,23324-2614,10/19/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,225134,Truist Bank d/b/a Branch Banking & Trust Co,214 N Tryon St,CHARLOTTE,NC,28202-1078,U,Y,Y,Existing or more than 2 years old,CHESAPEAKE,CHESAPEAKE CITY,VA,23324-2614,VA-03,500.0,336611.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,225134,Truist Bank d/b/a Branch Banking & Trust Co,CHARLOTTE,NC,Unanswered,Unanswered,NaN,10141944.44,9/20/21,33.0,1%,100000.0,1586476800,1632096000,Manufacturing,RICHMOND DISTRICT OFFICE,e $5-10 million,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,5.155e+14
1,1,1,896119,2649457102,4/11/20,304,PPP,"FIRE & LIFE SAFETY AMERICA, INC.",8827 STAPLES MILL RD,HENRICO,VA,23228-2014,10/21/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,60009,First Commonwealth Bank,600 Philadelphia St,INDIANA,PA,15701-3904,U,N,N,Existing or more than 2 years old,HENRICO,HENRICO,VA,23228-2014,VA-07,500.0,922160.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,60009,First Commonwealth Bank,INDIANA,PA,Unanswered,Unanswered,NaN,10143835.62,9/28/21,92.0,1%,100000.0,1586563200,1632787200,Public Administration,RICHMOND DISTRICT OFFICE,e $5-10 million,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,5.10872e+14
2,2,2,896120,2952437110,4/11/20,304,PPP,"TIMMONS GROUP, INC.",1001 Boulders Parkway Ste 300,RICHMOND,VA,23225-5505,8/5/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,123548,Towne Bank,5716 High St West,PORTSMOUTH,VA,23703-4502,U,Y,Y,Existing or more than 2 years old,RICHMOND,RICHMOND CITY,VA,23225-5505,VA-07,500.0,541330.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,123548,Towne Bank,PORTSMOUTH,VA,Unanswered,Unanswered,NaN,10124722.22,7/14/21,54.0,1%,100000.0,1586563200,1626220800,"Professional, Scientific, and Technical Services",RICHMOND DISTRICT OFFICE,e $5-10 million,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,5.10411e+14
3,3,3,896121,3379937304,4/29/20,353,PPP,LOUDOUN MEDICAL GROUP PC,224 D Cornwall Street Suite 403,LEESBURG,VA,20176,10/7/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,44449,"PNC Bank, National Association",222 Delaware Ave,WILMINGTON,DE,19801-1621,U,N,N,Unanswered,LEESBURG,LOUDOUN,VA,20176-1200,VA-10,500.0,621112.0,Unanswered,Unknown/NotStated,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,Corporation,44449,"PNC Bank, National Association",WILMINGTON,DE,Unanswered,Unanswered,NaN,10137777.78,9/10/21,62.0,1%,100000.0,1588118400,1631232000,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,e $5-10 million,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,5.11076e+14
4,4,4,896122,3773127208,4/27/20,304,PPP,BURGERBUSTERS INC,2242 W GREAT NECK RD,VIRGINIA BEACH,VA,23451,7/21/21,Paid in Full,24,100,10000000.0,10000000.0,0.0,NaN,44449,"PNC Bank, National Association",222 Del

In [95]:
null_check = geo_va_fips.loc[geo_va_fips["FIPS_z"].isnull()] #13 null 

In [4]:
geo_va_fips.to_csv("data/state_data/geo/geo_fips/va_fips_scraped.csv") 

In [4]:
pd.set_option('float_format', '{:f}'.format)
geo_va_fips = pd.read_csv("data/state_data/geo/geo_fips/va_fips_scraped.csv") 
geo_va_fips = geo_va_fips.drop(["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1)
geo_va_fips

,index,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,Estimated_LenderProfit,DateApproved_ts,ForgivenessDate_ts,Industry,SBAOfficeLabel,Loan_Range,full_add,Lat,Long,FIPS_z
0,896118,2392497107,4/10/20,304,PPP,TECNICO CORPORATION,831 INDUSTRIAL AVE,CHESAPEAKE,VA,23324-2614,...,100000.000000,1586476800,1632096000,Manufacturing,RICHMOND DISTRICT OFFICE,e $5-10 million,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,515500203001000.000000
1,896119,2649457102,4/11/20,304,PPP,"FIRE & LIFE SAFETY AMERICA, INC.",8827 STAPLES MILL RD,HENRICO,VA,23228-2014,...,100000.000000,1586563200,1632787200,Public Administration,RICHMOND DISTRICT OFFICE,e $5-10 million,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,510872005033029.000000
2,896120,2952437110,4/11/20,304,PPP,"TIMMONS GROUP, INC.",1001 Boulders Parkway Ste 300,RICHMOND,VA,23225-5505,...,100000.000000,1586563200,1626220800,"Professional, Scientific, and Technical Services",RICHMOND DISTRICT OFFICE,e $5-10 million,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,510411001071014.000000
3,896121,3379937304,4/29/20,353,PPP,LOUDOUN MEDICAL GROUP PC,224 D Cornwall Street Suite 403,LEESBURG,VA,20176,...,100000.000000,1588118400,1631232000,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,e $5-10 million,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,511076104001030.000000
4,896122,3773127208,4/27/20,304,PPP,BURGERBUSTERS INC,2242 W GREAT NECK RD,VIRGINIA BEACH,VA,23451,...,100000.000000,1587945600,1623369600,Accomodation and Food Services,RICHMOND DISTRICT OFFICE,e $5-10 million,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,518100430061015.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23263,919381,9512327001,4/9/20,304,PPP,"PERRY POOLS, INC.",6000 FORT AVE,LYNCHBURG,VA,24502-1932,...,7500.000000,1586390400,1617580800,Construction,RICHMOND DISTRICT OFFICE,"a $150,000-350,000","6000 FORT AVE, LYNCHBURG, VA",37.371059,-79.194838,516800009013012.000000
23264,919382,9677417002,4/9/20,304,PPP,GRACE COVENANT CHURCH,3337 Emmaus Rd,ROCKINGHAM,VA,22801-2685,...,7500.000000,1586390400,1620777600,Other Services (except Public Administration),RICHMOND DISTRICT OFFICE,"a $150,000-350,000","3337 Emmaus Rd, ROCKINGHAM, VA",38.406665,-78.913133,511650114005035.000000
23265,919383,9715167306,5/2/20,304,PPP,METRO PRECAST AND STONE SERVICES INC,8503 BUCYRUS CT,MANASSAS,VA,20110-5321,...,7500.000000,1588377600,1608681600,Construction,RICHMOND DISTRICT OFFICE,"a $150,000-350,000","8503 BUCYRUS CT, MANASSAS, VA",38.761194,-77.454184,516839103011000.000000
23266,919384,9767568404,2/17/21,304,PPS,AMES CLEANERS & FORMALS INC,554 E Mercury Blvd,Hampton,VA,23663-2229,...,7500.000000,1613520000,1634688000,Retail Trade,RICHMOND DISTRICT OFFICE,"a $150,000-350,000","554 E Mercury Blvd, Hampton, VA",37.022287,-76.319882,516500109001011.000000


In [5]:
geo_va_fips = pd.read_csv('data/state_data/geo/geo_fips/va_fips_scraped.csv')
geo_va_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,index,LoanNumber,DateApproved,SBAOfficeCode,...,Estimated_LenderProfit,DateApproved_ts,ForgivenessDate_ts,Industry,SBAOfficeLabel,Loan_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,0,896118,2392497107,4/10/20,304,...,100000.0,1586476800,1632096000,Manufacturing,RICHMOND DISTRICT OFFICE,e $5-10 million,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,5.155002e+14
1,1,1,1,1,1,1,896119,2649457102,4/11/20,304,...,100000.0,1586563200,1632787200,Public Administration,RICHMOND DISTRICT OFFICE,e $5-10 million,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,5.108720e+14
2,2,2,2,2,2,2,896120,2952437110,4/11/20,304,...,100000.0,1586563200,1626220800,"Professional, Scientific, and Technical Services",RICHMOND DISTRICT OFFICE,e $5-10 million,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,5.104110e+14
3,3,3,3,3,3,3,896121,3379937304,4/29/20,353,...,100000.0,1588118400,1631232000,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,e $5-10 million,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,5.110761e+14
4,4,4,4,4,4,4,896122,3773127208,4/27/20,304,...,100000.0,1587945600,1623369600,Accomodation and Food Services,RICHMOND DISTRICT OFFICE,e $5-10 million,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,5.181004e+14


In [7]:
geo_va_fips.loc[geo_va_fips["FIPS_z"].isnull()]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,index,LoanNumber,DateApproved,SBAOfficeCode,...,Estimated_LenderProfit,DateApproved_ts,ForgivenessDate_ts,Industry,SBAOfficeLabel,Loan_Range,full_add,Lat,Long,FIPS_z
328,328,328,328,328,328,328,896446,6940727104,4/14/20,304,...,31608.0000,1586822400,1623974400,Administrative and Support and Waste Managemen...,RICHMOND DISTRICT OFFICE,d $2-5 million,"4625 Creekstone Drive Suite 130, OAKPARK, VA",35.882469,-78.846699,NaN
2486,2486,2486,2486,2486,2486,2486,898604,3320167707,5/1/20,304,...,30566.1900,1588291200,-9223372037,Real Estate Rental and Leasing,RICHMOND DISTRICT OFFICE,c $1-2 million,"777 6th Street NW, WASHINGTON, VA",38.899370,-77.019590,NaN
4384,4384,4384,4384,4384,4384,4384,900502,3594887305,4/29/20,304,...,20100.6600,1588118400,1617235200,Retail Trade,RICHMOND DISTRICT OFFICE,"b $350,000-1 million","1120 Crain Highway North, Chesapeake, VA",39.181778,-76.615519,NaN
4520,4520,4520,4520,4520,4520,4520,900638,4204237701,5/1/20,304,...,19630.8000,1588291200,1623888000,Other Services (except Public Administration),RICHMOND DISTRICT OFFICE,"b $350,000-1 million","55 Joy Drive, South Burlington, VA",44.451222,-73.199114,NaN
5026,5026,5026,5026,5026,5026,5026,901144,2920497210,4/16/20,304,...,17910.0000,1586995200,1607472000,Health Care and Social Assistance,RICHMOND DISTRICT OFFICE,"b $350,000-1 million","1 Spring Street, HOT SPRINGS, VA",34.506051,-93.019824,NaN
7013,7013,7013,7013,7013,7013,7013,903131,9149677006,4/9/20,304,...,13656.9300,1586390400,1611100800,Educational Services,RICHMOND DISTRICT OFFICE,"b $350,000-1 million","4 Wallace Manor Road, BROAD RUN, VA",38.963740,-76.549139,NaN
7859,7859,7859,7859,7859,7859,7859,903977,9682888407,2/17/21,304,...,12420.0000,1613520000,-9223372037,"Professional, Scientific, and Technical Services",RICHMOND DISTRICT OFFICE,"b $350,000-1 million","995 RESEARCH BLVD STE 118, CHARLOTTESVILLE, VA",30.479120,-97.802932,NaN
7955,7955,7955,7955,7955,7955,7955,904073,1800157203,4/15/20,304,...,12303.0000,1586908800,1623369600,"Professional, Scientific, and Technical Services",RICHMOND DISTRICT OFFICE,"b $350,000-1 million","995 RESEARCH BLVD STE 118, CHARLOTTESVILLE, VA",30.479120,-97.802932,NaN
9704,9704,9704,9704,9704,9704,9704,905822,9453327109,4/15/20,304,...,17150.0000,1586908800,1623888000,Construction,RICHMOND DISTRICT OFFICE,"a $150,000-350,000","5518 West Linebaugh Avenue, CHESTER, VA",28.039440,-82.536945,NaN
11529,11529,11529,11529,11529,11529,11529,907647,1573407107,4/10/20,304,...,14735.0000,1586476800,1617148800,Construction,RICHMOND DISTRICT OFFICE,"a $150,000-350,000","4417 S Sheridan Rd, CHURCH ROAD, VA",36.098468,-95.903809,NaN


In [8]:
geo_va_fips.to_csv("data/state_data/geo/geo_fips/VA_fips_scraped.csv") 